This is draft code for uploading spreadsheet data on subjects into custom fields in XNAT.

The code will set up your XNAT connection and then prompt you to select the csv file.

To Do: create custom fields based on column headers, insert into existing fields for subjectData (gender, etc..), get mset working to speed up process (currently nested for loops)?

Date, Comments/Edits, Author
08-29-17, Initial draft, JH

In [ ]:
#clear
%reset

In [48]:
#import libraries

import pyxnat
from pyxnat import Interface

import os
from os import listdir
from os.path import isfile, join

from  Tkinter import *
import Tkinter, Tkconstants, tkFileDialog

import pandas as pd


In [49]:
#setup xnat connection

connectSuccess = False

while not connectSuccess:
    xnat = Interface(server='http://xnat.bidmc.harvard.edu:8080', cachedir='/tmp')
    listProjects = xnat.select.projects().get()
    
    if not listProjects:
        
        print 'Connection unsuccessful, try again.'
        
    else:
        
        print 'Connection successful, available projects are: '
        print listProjects
        connectSuccess = True


User: jhsu3
········
Connection successful, available projects are: 
['TMS_DEPRESSION', 'GrafmanLesion', 'Test', 'DBS', 'Corbetta', 'Aging', 'DIABETES']


In [93]:
#select or create new project

exitProjSelect = False

#keep prompting user for project selection
while not exitProjSelect:
    
    #enter project name
    projectName = raw_input('Select project: ')
    projectObj = xnat.select.project(projectName)
    
    #if project entered doesn't exist, ask if want to create
    if not projectObj.exists():
    
        print 'Project does not exist.'
        createProj = raw_input('Create new project? (y/n/anything else to exit) ')
    
        #create new project
        if createProj == 'y':
            
            projectObj.create()
            exitProjSelect = True
            print 'Created new project.'
        
        #go back to project selection if no
        elif createProj == 'n':

            print 'Try again.\n'
        
        #exit project selection
        else:
            
            exitProjSelect = True
            print 'Exiting selection.'
    
    #project exists
    else:
        
        exitProjSelect = True
        print 'Project exists and selected.'
        



Select project: MGH_Lesions
Project exists and selected.


In [134]:
#select csv file with data for custom variables

#bring up GUI to select directory
print('Select csv file with custom variables.')

root = Tk()
root.update()

csvFilePath = tkFileDialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("csv files","*.csv"),("all files","*.*")))

root.destroy()

df = pd.read_csv(csvFilePath)
print('Here are the first two rows of the csv file: \n')
df.loc[0:1,:]

Select csv file with custom variables.
Here are the first two rows of the csv file: 



,Filename,mrs,nihss
0,subject001,2,3
1,subject002,2,3


In [130]:
ID_column = input('What is the index of the column containing subject IDs? ')

print('Here is an example: ' + df.iloc[0,ID_column])

exitSubjNumSelect = False 
print('Indicate the indices for the subject ID.')

while not exitSubjNumSelect:

    firstInd = input('First index: ')
    secondInd = input('Second index: ')

    if secondInd <= firstInd:

        print 'Error, second index must be larger than first. Try again.\n'

    else:

        print('Here is what you selected: ' + df.iloc[0,ID_column][firstInd:secondInd])
        subjNumCorrect = raw_input('Is this correct? (y/n/anything else to exit): ')

        if subjNumCorrect == 'y':

            exitSubjNumSelect = True

        elif subjNumCorrect == 'n':

            print 'Try again\n'

        else:

            exitSubjNumSelect = True

What is the index of the column containing subject IDs? 0
Here is an example: subject001
Indicate the indices for the subject ID.
First index: 7
Second index: 10
Here is what you selected: 001
Is this correct? (y/n/anything else to exit): y


In [136]:
for j in range(0,len(df.columns)):
    
#     print j
    
    if j == ID_column:
        
        continue
    
    else:
        
        fieldName = df.columns[j]
        fieldPath = "xnat:subjectData/fields/field[name='%s']/field" %fieldName.lower()
        
        print 'Uploading data for field: ' + fieldName
#         print fieldPath
        for i in range(0,len(df.iloc[:,0])):
            
#             print i

            subjectID = df.iloc[i,ID_column][firstInd:secondInd]
            subjectObj = projectObj.subject(subjectID)
            
            print subjectObj
            print df.iloc[i,j]
            
            subjectObj.attrs.set(fieldPath,df.iloc[i,j]) 
            subjectObj.create()
            

Uploading data for field: mrs
<Subject Object> 001
2


AttributeError: 'numpy.int64' object has no attribute 'rstrip'